https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [1]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [2]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 0
%matplotlib inline
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from datetime import datetime

from helpers.feature_engineering import dateStrToDayYear, getMappingOfSimilarCategoricalColumns, \
    compareSimilarCategoricalColumns 
from helpers.my_one_hot_encoder import MyOneHotEncoder

In [4]:
rng = np.random.RandomState(seed=random_state)

# Flying to New York City - Target Classes Preprocessing

In [5]:
path_data = os.path.realpath(os.path.join(os.getcwd(), '../Data', 'binary_train_data_numerical.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/binary_train_data_numerical.csv'

In [6]:
df = pd.read_csv(path_data, delimiter = ',', header=0)
df.shape

(186308, 533)

### Feature Date

In [7]:
df[:5]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,...,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED
0,2016,3,7,3,7,2016-07-03,HA,19690,HA,50,...,0.0,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False
1,2016,2,4,2,6,2016-04-02,B6,20409,B6,1304,...,8.0,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False
2,2016,1,3,26,6,2016-03-26,B6,20409,B6,164,...,43.0,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True
3,2016,1,1,13,3,2016-01-13,B6,20409,B6,429,...,0.0,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False
4,2016,4,12,18,7,2016-12-18,DL,19790,DL,786,...,0.0,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False


The year attribute is currently the same year, so we could just remove it.
While the flight date attribute brings little information extra information. It could be more useful to convert it to the day of the year since some days could have some particular on a yearly basis.
Although we understand that this attribute will have a correlation with the Quarter, Month, and Day of Month attributes.

In [8]:
df['FL_DATE'].values.shape#[np.newaxis].shape

(186308,)

In [9]:
df['YDAY'] = [dateStrToDayYear(dateStr) for dateStr in df['FL_DATE']]

In [10]:
df[:5]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,...,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED,YDAY
0,2016,3,7,3,7,2016-07-03,HA,19690,HA,50,...,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False,185
1,2016,2,4,2,6,2016-04-02,B6,20409,B6,1304,...,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False,93
2,2016,1,3,26,6,2016-03-26,B6,20409,B6,164,...,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True,86
3,2016,1,1,13,3,2016-01-13,B6,20409,B6,429,...,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False,13
4,2016,4,12,18,7,2016-12-18,DL,19790,DL,786,...,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False,353


In [11]:
df_1 = df.drop(labels=['YEAR', 'FL_DATE'], axis=1)
df_1.shape

(186308, 39)

In [12]:
df_1[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED,YDAY
0,3,7,3,7,HA,19690,HA,50,12173,HNL,...,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False,185
1,2,4,2,6,B6,20409,B6,1304,14843,SJU,...,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False,93
2,1,3,26,6,B6,20409,B6,164,14986,SRQ,...,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True,86
3,1,1,13,3,B6,20409,B6,429,12478,JFK,...,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False,13
4,4,12,18,7,DL,19790,DL,786,12953,LGA,...,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False,353
